In [225]:
import pickle
noise_index = 9

# iterate over all the files in folder
folderName = f'/home//StepByStep/InformationFlow/result/{noise_index}'

In [226]:

def nested_data(data, entry):
    nested_dict = {}
    if(entry not in data):
        print("Not present")
        return None
    for d in data[entry]:
        nested_dict[d] = nested_data(data, d)
    return nested_dict





In [227]:
def getNestedData(data):
    clean_data = {}
    for d in data:
        attend = []
        for d1 in d['attending_layer_head']:
            attend.append(f'layer_{d1["layer"]}_head_{d1["head"]}_pos_{d1["pos"]}')
        clean_data[f'layer_{d["layer"]}_head_{d["head"]}_pos_{d["pos"]}'] = attend
    depth_0_entry = []
    for d in data:
        if d['depth'] == 0:
            depth_0_entry.append(f'layer_{d["layer"]}_head_{d["head"]}_pos_{d["pos"]}')
    nested = {}

    for entry in depth_0_entry:
        nested[entry] = nested_data(clean_data, entry)
    return nested

In [228]:
max_depth = 0
totalExamples = 0
import os
complete_data = []
for filename in os.listdir(folderName):
    layer_head_depth_data = []
    if(filename.endswith('.pickle') and filename.startswith('example')):
        with open(f'{folderName}/{filename}', 'rb') as f:
            data = pickle.load(f)
        # print(data)

        for d in data:
            if((d['layer'], d['head'], d['depth']) not in layer_head_depth_data):
                layer_head_depth_data.append((d['layer'], d['head'], d['depth']))
            for attending in d['attending_layer_head']:
                if((attending['layer'], attending['head'], d['depth'] + 1) not in layer_head_depth_data):
                    layer_head_depth_data.append((attending['layer'], attending['head'], d['depth'] + 1))
                    max_depth = max(max_depth, d['depth'] + 1)
        complete_data.extend(layer_head_depth_data)
        totalExamples += 1
    # break


In [229]:
freq_layer_head_depth = {}
for data in complete_data:
    if(data in freq_layer_head_depth):
        freq_layer_head_depth[data] += 1
    else:
        freq_layer_head_depth[data] = 1

In [230]:
# with open(f'{folderName}/layer_head_depth.pickle', 'wb') as f:
#     pickle.dump(freq_layer_head_depth, f)

In [231]:
# depth_0_layer_head = []
# for noise_index in range(10):
#     layer_head = []
#     with open(f'/home//StepByStep/InformationFlow/result/{noise_index}/layer_head_depth.pickle', 'rb') as f:
#         data = pickle.load(f)
#     for d in data:
#         if((d[0], d[1]) not in layer_head and d[2] == 0):
#             layer_head.append((d[0], d[1]))
#     depth_0_layer_head.append(layer_head)
    

In [232]:
# import matplotlib.pyplot as plt

# # Example: 10 lists of tuples with x and y coordinates
# # data_lists = [
#     # [(1, 2), (3, 4), (5, 6)],
#     # [(2, 3), (4, 5), (6, 7)],
#     # [(1, 5), (2, 6), (3, 7)],
#     # ... add 7 more lists
# # ]

# # Define 10 different colors
# colors = ['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#FF00FF', '#00FFFF', '#FFA500', '#800080', '#008000', '#000080']

# # Create a scatter plot with different colors for each list
# for i, data_list in enumerate(depth_0_layer_head):
#     x_vals, y_vals = zip(*data_list)
#     plt.scatter(x_vals, y_vals, label=f'Index {i}', color=colors[i], s=30)

# # Set axis limits
# plt.xlim(min(x for sublist in depth_0_layer_head for x, _ in sublist) - 1, max(x for sublist in depth_0_layer_head for x, _ in sublist) + 1)
# plt.ylim(min(y for sublist in depth_0_layer_head for _, y in sublist) - 1, max(y for sublist in depth_0_layer_head for _, y in sublist) + 1)

# # Add legend
# plt.legend()
# plt.gca().invert_yaxis()
# # plt.gca().invert_xaxis()
# plt.xlabel('Head')
# plt.ylabel('Layer')
# # Show the plot
# plt.show()


In [233]:
import transformer_lens.utils as utils
import plotly.express as px
def imshow(tensor, renderer=None, save='head.svg', **kwargs):
    fig = px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs)
    fig.show(renderer)
    # save svg
    # fig.savefig(save)
    fig.write_html(save)

In [234]:
matrixes = []
for depth in range(4):
    filtered = {}
    for data in freq_layer_head_depth:
        if(data[2] == depth):
            filtered[data] = freq_layer_head_depth[data] / totalExamples
    distribution = []
    for layer in range(32):
        temp = []
        for head in range(32):
            temp.append(0)
        distribution.append(temp)
    # iterate over dictionary
    for data in filtered:
        distribution[data[0]][data[1]] = filtered[data]
    imshow(distribution, save=f'/home//StepByStep/InformationFlow/result/{noise_index}/depth_{depth}.html', title=f"index {noise_index}, depth {depth}", labels={"x":"Head", "y":"Layer"})
    import json
    import re
    import plotly.io

    # Assuming html_file_path is the path to your HTML file
    with open(f'/home//StepByStep/InformationFlow/result/{noise_index}/depth_{depth}.html') as f:
        html = f.read()

    # Extracting the Plotly JSON from the HTML
    call_arg_str = re.findall(r'Plotly\.newPlot\((.*)\)', html[-2**16:])[0]
    call_args = json.loads(f'[{call_arg_str}]')
    plotly_json = {'data': call_args[1], 'layout': call_args[2]}    

    # Creating a Plotly figure from the JSON
    fig = plotly.io.from_json(json.dumps(plotly_json))

    # Specify the output file path with the desired format (e.g., 'figure.pdf' or 'figure.svg')
    output_file_path = f'/home//StepByStep/InformationFlow/result/{noise_index}/depth_{depth}_index_{noise_index}.svg'

    # Save the figure as PDF or SVG
    plotly.io.write_image(fig, output_file_path)


    
        


In [116]:
depth = 0


In [118]:
import transformer_lens.utils as utils
import plotly.express as px
def imshow(tensor, renderer=None, save='head.html', **kwargs):
    fig = px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs)
    fig.show(renderer)
    # fig.write_html(save)

In [133]:
import torch
import transformer_lens.utils as utils

# Example: Four 32x32 matrices
matrix1 = torch.rand(32, 32)
matrix2 = torch.rand(32, 32)
matrix3 = torch.rand(32, 32)
matrix4 = torch.rand(32, 32)

# Assuming `imshow` is defined in the utils module
def imshow(tensor, renderer=None, save='head.html', **kwargs):
    import plotly.express as px  # Importing inside the function for better encapsulation
    fig = px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs)
    fig.show(renderer)
    # fig.write_html(save)

# Plot each matrix using imshow
imshow(matrix1, save='matrix1.html', title='Matrix 1')
imshow(matrix2, save='matrix2.html', title='Matrix 2')
imshow(matrix3, save='matrix3.html', title='Matrix 3')
imshow(matrix4, save='matrix4.html', title='Matrix 4')


In [131]:
imshow(matrixes[0], save=f'', title=f"Depth {depth}", labels={"x":"Head", "y":"Layer"})


In [132]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Example: Four 32x32 matrices
matrix1 = np.random.rand(32, 32)
matrix2 = np.random.rand(32, 32)
matrix3 = np.random.rand(32, 32)
matrix4 = np.random.rand(32, 32)

# Define colors for each matrix
color_scales = ['Viridis', 'YlGnBu', 'RdYlBu', 'Jet']

# Create subplots with shared axes
fig = make_subplots(rows=2, cols=2, subplot_titles=['Matrix 1', 'Matrix 2', 'Matrix 3', 'Matrix 4'])

# Add trace for each matrix
for i, matrix in enumerate([matrix1, matrix2, matrix3, matrix4]):
    row = i // 2 + 1
    col = i % 2 + 1

    fig.add_trace(
        go.Heatmap(
            z=matrix,
            colorscale=color_scales[i],
            showscale=False,
        ),
        row=row,
        col=col,
    )

# Update layout
fig.update_layout(
    height=800,
    width=800,
    title_text='Combined Plots',
)

# Show the plot
fig.show()
